In [1]:
import mysql.connector as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#merge tables to one table
def createTable(players,stats):
    players['Player_ID'] = players['Player_ID'].astype(int)
    stats['Player_ID'] = stats['Player_ID'].astype(int)
    player_stats = pd.merge(players, stats.drop_duplicates(), on=['Player_ID'])
    player_stats.reset_index(drop=True)

    #create age column
    player_stats['age'] = (player_stats['Year']).astype('int64') - player_stats['birth_year']

    #drop unesessary columns
    stats = player_stats.drop([u'height', u'weight', u'college', u'round_drafted',u'draft_pick',u'draft_year', u'current_salary'], axis=1)
    return stats

In [3]:
# # def getStatsByPosition(x):
def getQBStats(stats):
    QB = pd.DataFrame(stats[stats['position'] == "QB"])
    QB = QB.drop([u'Team',                 u'game_won',
              u'player_team_score',           u'opponent_score',
                       u'passing_rating',
                  u'passing_sacks', u'passing_sacks_yards_lost',
               u'receiving_targets',
           u'receiving_receptions',          u'receiving_yards',
           u'receiving_touchdowns',     u'kick_return_attempts',
              u'kick_return_yards',   u'kick_return_touchdowns',
           u'point_after_attempts',        u'point_after_makes',
            u'field_goal_attempts',         u'field_goal_makes'], axis =1)
    return QB

def getRBStats(stats):
    RB = pd.DataFrame(stats[stats['position'] == "RB"])
    RB = RB.drop([u'Team',                 u'game_won',
              u'player_team_score',           u'opponent_score',
               u'passing_attempts',      u'passing_completions',
                  u'passing_yards',           u'passing_rating',
             u'passing_touchdowns',    u'passing_interceptions',
                  u'passing_sacks', u'passing_sacks_yards_lost',
                u'kick_return_attempts',
              u'kick_return_yards',   u'kick_return_touchdowns',
           u'point_after_attempts',        u'point_after_makes',
            u'field_goal_attempts',         u'field_goal_makes'], axis = 1)
    return RB

def getWRStats(stats):
    WR = pd.DataFrame(stats[stats['position'] == "WR"])
    WR = WR.drop([u'Team',                 u'game_won',
              u'player_team_score',           u'opponent_score',
               u'passing_attempts',      u'passing_completions',
                  u'passing_yards',           u'passing_rating',
             u'passing_touchdowns',    u'passing_interceptions',
                  u'passing_sacks', u'passing_sacks_yards_lost',
         u'kick_return_attempts',
              u'kick_return_yards',   u'kick_return_touchdowns',
           u'point_after_attempts',        u'point_after_makes',
            u'field_goal_attempts',         u'field_goal_makes'], axis = 1)
    return WR

def getTEStats(stats):
    TE = pd.DataFrame(stats[stats['position'] == "TE"])
    TE = TE.drop([u'Team',                 u'game_won',
              u'player_team_score',           u'opponent_score',
               u'passing_attempts',      u'passing_completions',
                  u'passing_yards',           u'passing_rating',
             u'passing_touchdowns',    u'passing_interceptions',
                  u'passing_sacks', u'passing_sacks_yards_lost',
               u'rushing_attempts',            u'rushing_yards',
             u'rushing_touchdowns',      u'kick_return_attempts',
              u'kick_return_yards',   u'kick_return_touchdowns',
           u'point_after_attempts',        u'point_after_makes',
            u'field_goal_attempts',         u'field_goal_makes'], axis = 1)
    return TE

def getKStats(stats):
    K = pd.DataFrame(stats[stats['position'] == "K"])
    K = K.drop([u'Team',                 u'game_won',
              u'player_team_score',           u'opponent_score',
               u'passing_attempts',      u'passing_completions',
                  u'passing_yards',           u'passing_rating',
             u'passing_touchdowns',    u'passing_interceptions',
                  u'passing_sacks', u'passing_sacks_yards_lost',
               u'rushing_attempts',            u'rushing_yards',
             u'rushing_touchdowns',        u'receiving_targets',
           u'receiving_receptions',          u'receiving_yards',
           u'receiving_touchdowns',     u'kick_return_attempts',
              u'kick_return_yards',   u'kick_return_touchdowns'], axis = 1)
    return K

# notes

Class
1 < 50% - bad
3 50% <= x < 75% ok
4  >= 75% great

*Interceptions it is reversed

Columns
Class | Age | stats


In [4]:
#gives player value a class value
def set_class(x):
    cl = []
    
    class1 = x.iloc[:,1].mean() / 2
    class3 = x.iloc[:,1].mean() + class1

    for i in x.iloc[:,1]:
        if( i < class1):
            cl.append(1)
        elif((i >= class1) and (i < class3)):
            cl.append(2)
        else:
            cl.append(3)
    classes = pd.Series(cl)
    x = x.assign(classes = classes.values)
    return x

In [5]:
#probability of each class. 
def class_probability(x):
    total = len(x)
    c1 = 0.0
    c2 = 0.0
    c3 = 0.0
    #get cl
    for i in x.iloc[:,2]:
        if( i == 1):
            c1 = c1 + 1.0
        elif(i == 2):
            c2 = c2 + 1.0
        else:
            c3 = c3 + 1.0
    pc1 = c1/total
    pc2 = c2/total
    pc3 = c3/total
    
    return [c1,c2,c3, pc1, pc2, pc3]

#Get the probability's for each class by age
def age_prob(x):
    c1,c2,c3, pc1, pc2, pc3  = class_probability(x)
    ages = []
    
    cl1 = x[x['classes'] == 1]
    cl2 = x[x['classes'] == 2]
    cl3 = x[x['classes'] == 3]
    
    one = []
    two = []
    three = []
    p1 = []
    p2 = []
    p3 = []
    
    for i in x.iloc[:,0]:
        ages.append(i)
    ages = np.unique(ages)
    
    for j in ages:
        for i in cl1['age']:
            if (j == i):
                one.append(j)  
                
    for j in ages:
        for i in cl2['age']:
            if (j == i):
                two.append(j) 
                
    for j in ages:
        for i in cl3['age']:
            if (j == i):
                three.append(j)
                
    for i in ages:
        if c1 == 0:
            p1.append(0)
        else:
            p1.append((pc1 * (one.count(i) / c1)))
    for i in ages:
        if c2 == 0:
            p2.append(0)
        else:    
            p2.append(pc2 * (two.count(i) / c2))
    for i in ages:
        if c3 == 0:
            p3.append(0)
        else:    
            p3.append(pc3 * (three.count(i) / c3))
        
        
        
    l = {'Age': ages, 'Probability of Class 1': p1, 'Probability of Class 2': p2, 'Probability of Class 3': p3}
    class_prob = pd.DataFrame(l)    
    
    return class_prob


In [6]:
#match age with winning class
def ageToClass(x):
    probl = []
    age = []
    c1 = 1
    c2 = 2
    c3 = 3
    
    for i in range(len(x)):
        age.append(x.iloc[i,0])
        if(x.iloc[i,1] > x.iloc[i,2]):
            if(x.iloc[i,1] > x.iloc[i,3]):
                probl.append(c1)
            else:
                 probl.append(c3)
        elif(x.iloc[i,2] > x.iloc[i,1]):
            if(x.iloc[i,2] > x.iloc[i,3]):
                probl.append(c2)
            else:
                 probl.append(c3)
        else:
            probl.append(c2)
    
    l = { 'Age': age, 'Class' : probl}
    
    return pd.DataFrame(l)
        


In [7]:
#Classifying each age by what class they are

def QBProbability(QB):
    QBpassing = pd.DataFrame(QB.filter(['age','passing_yards'],axis=1))
    QBpassing['passing_yards'] = QBpassing['passing_yards'].astype(float)
    QBpassing = QBpassing[QBpassing['passing_yards'] > 1000]
    QBpassing = set_class(QBpassing)
    pPassing = age_prob(QBpassing)
    p_py = ageToClass(pPassing)


    QBTD = pd.DataFrame(QB.filter(['age','passing_touchdowns']))
    QBTD['passing_touchdowns'] = QBTD['passing_touchdowns'].astype(float)
    QBTD = QBTD[QBTD['passing_touchdowns'] > 10]
    QBTD = set_class(QBTD) 
    pTD = age_prob(QBTD)
    p_TD = ageToClass(pTD)


    QBInt = pd.DataFrame(QB.filter(['age', 'passing_interceptions']))
    QBInt['passing_interceptions'] = QBInt['passing_interceptions'].astype(float)
    QBTD = QBInt[QBInt['passing_interceptions'] > 0]
    QBInt = set_class(QBInt)
    pINT = age_prob(QBInt)
    p_INT = ageToClass(pINT)

    l = {"Age" : p_py.iloc[:,0], "QB Passing" : p_py.iloc[:,1], "QB Touchdowns" : p_TD.iloc[:,1], "QB Interceptions" : p_INT.iloc[:,1]}
    QB_Probability = pd.DataFrame(l)
    return QB_Probability

def RBProbability(RB):
    RByards = pd.DataFrame(RB.filter(['age','rushing_yards'],axis=1))
    RByards['rushing_yards'] = RByards['rushing_yards'].astype(float)
    RByards = RByards[RByards['rushing_yards'] > 150]
    RByards = set_class(RByards)
    pyards = age_prob(RByards)
    p_yards = ageToClass(pyards)

    RBTD = pd.DataFrame(RB.filter(['age','rushing_touchdowns']))
    RBTD['rushing_touchdowns'] = RBTD['rushing_touchdowns'].astype(float)
    RBTD = RBTD[RBTD['rushing_touchdowns'] > 1]
    RBTD = set_class(RBTD)
    pTD = age_prob(RBTD)
    p_TD = ageToClass(pTD)

    l = {"Age" : p_TD.iloc[:,0], "RB Rushing Yards" : p_yards.iloc[:,1], "RB Touchdowns" : p_TD.iloc[:,1]}
    RB_Probability = pd.DataFrame(l)
    return RB_Probability

def WRProbability(WR):
    WRyards = pd.DataFrame(WR.filter(['age','receiving_yards'],axis=1))
    WRyards['receiving_yards'] = WRyards['receiving_yards'].astype(float)
    WRyards = WRyards[WRyards['receiving_yards'] > 150]
    WRyards = set_class(WRyards)
    pyards = age_prob(WRyards)
    p_yards = ageToClass(pyards)

    WRTD = pd.DataFrame(WR.filter(['age','receiving_touchdowns'],axis=1))
    WRTD['receiving_touchdowns'] = WRTD['receiving_touchdowns'].astype(float)
    WRTD = WRTD[WRTD['receiving_touchdowns'] > 1]
    WRTD = set_class(WRTD)
    pTD = age_prob(WRTD)
    p_TD = ageToClass(pTD)

    l = {"Age" : p_TD.iloc[:,0], "WR Receiving Yards" : p_yards.iloc[:,1], "WR Touchdowns" : p_TD.iloc[:,1]}
    WR_Probability = pd.DataFrame(l)
    return WR_Probability

def TEProbability(TE):
    TEyards = pd.DataFrame(TE.filter(['age','receiving_yards'],axis=1))
    TEyards['receiving_yards'] = TEyards['receiving_yards'].astype(float)
    TEyards = TEyards[TEyards['receiving_yards'] > 150]
    TEyards = set_class(TEyards)
    pYards = age_prob(TEyards)
    p_yards = ageToClass(pYards)

    TETD = pd.DataFrame(TE.filter(['age','receiving_touchdowns'],axis=1))
    TETD['receiving_touchdowns'] =TETD['receiving_touchdowns'].astype(float)
    TETD = TETD[TETD['receiving_touchdowns'] > 1]
    TETD = set_class(TETD)
    pTD = age_prob(TETD)
    p_TD = ageToClass(pTD)

    l = {"Age" : p_TD.iloc[:,0], "TE Receiving Yards" : p_yards.iloc[:,1], "TE Touchdowns" : p_TD.iloc[:,1]}
    TE_Probability = pd.DataFrame(l)
    return TE_Probability

def KProbability(K):
    KPAT= pd.DataFrame(K.filter(['age','point_after_makes'],axis=1))
    KPAT['point_after_makes'] = KPAT['point_after_makes'].astype(float)
    KPAT = KPAT[KPAT['point_after_makes'] > 1]
    KPAT = set_class(KPAT)
    pPAT = age_prob(KPAT)
    p_PAT = ageToClass(pPAT)

    KFG = pd.DataFrame(K.filter(['age','field_goal_makes'],axis=1))
    KFG['field_goal_makes'] = KFG['field_goal_makes'].astype(float)
    KFG = KFG[KFG['field_goal_makes'] > 1]
    KFG = set_class(KFG)
    pKG = age_prob(KFG)
    p_KG = ageToClass(pKG)

    l = {"Age" : p_PAT.iloc[:,0], "K PAT" : p_PAT.iloc[:,1], "K Field Goals" : p_KG.iloc[:,1]}
    K_Probability = pd.DataFrame(l)
    return K_Probability

# Main
    *Connect to Database
    *create Queries and get data from database
    *Train
    *Test
    *Find efficinecy

In [8]:
cnx = sql.connect(user='root', password='', host='127.0.0.1', database='nfl_player_stats')

cursor = cnx.cursor()

player = ("SELECT * FROM players")
past = ("SELECT * FROM offense_stats")
# current = ("SELECT * FROM current_stats")

players_df = pd.read_sql(player, cnx)
past_df = pd.read_sql(past, cnx)
# current_df = pd.read_sql(current, cnx)

players_df = pd.DataFrame(players_df)
train_df = pd.DataFrame(past_df)
# test_df = pd.DataFrame(current_df)

cnx.close()


In [9]:
train = createTable(players_df,train_df)
train17 = train[train['Year'] != '2017']
train16 = train[train['Year'] != '2016']
train15 = train[train['Year'] != '2015']
test17 = train[train['Year'] == '2017']
test16 = train[train['Year'] == '2016']
test15 = train[train['Year'] == '2015']



# QB Prediction Accuracy

In [10]:
# train = createTable(players_df,train_df)
# test = createTable(players_df,test_df)
def QBTest(train,test):
    QBTrain = getQBStats(train)
    # print QBTrain['passing_interceptions'].mean() 
    # print QBTrain['passing_interceptions'].median() 
    QBProbability_Train = QBProbability(QBTrain)
#     print QBProbability_Train
    QBTest = getQBStats(test)
    # print QBTest['passing_interceptions'].mean() 
    # print QBTest['passing_interceptions'].median() 
    QBProbability_Test = QBProbability(QBTest)
#     print QBProbability_Test

    # print QBProbability_Test.columns
    l = []
    for i in range(len(QBProbability_Train.iloc[:,0])):
        for j in range(len(QBProbability_Test.iloc[:,0])):
            #age
                if(QBProbability_Train.iloc[i,0] == QBProbability_Test.iloc[j,0]):
                    #stat
                    if QBProbability_Train.iloc[i,1] == QBProbability_Test.iloc[j,1]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'QB Interception Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)

    l = []
    for i in range(len(QBProbability_Train.iloc[:,0])):
        for j in range(len(QBProbability_Test.iloc[:,0])):
            #age
                if(QBProbability_Train.iloc[i,0] == QBProbability_Test.iloc[j,0]):
                    #stat
                    if QBProbability_Train.iloc[i,2] == QBProbability_Test.iloc[j,2]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'QB Passing Accuacy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)

    l = []
    for i in range(len(QBProbability_Train.iloc[:,0])):
        for j in range(len(QBProbability_Test.iloc[:,0])):
            #age
                if(QBProbability_Train.iloc[i,0] == QBProbability_Test.iloc[j,0]):
                    #stat
                    if QBProbability_Train.iloc[i,3] == QBProbability_Test.iloc[j,3]:
                        l.append('y')
                    else:
                        l.append('f')
    T = float(l.count('y'))
    F = float(l.count('f') )

    print 'QB Touchdowns accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)
    
print QBTest(train17,test17)
print QBTest(train16,test16)
print QBTest(train15,test15)

QB Interception Accuracy: 
correct answers:  0.2
false answers:  0.8
QB Passing Accuacy: 
correct answers:  0.933333333333
false answers:  0.0666666666667
QB Touchdowns accuracy: 
correct answers:  1.0
false answers:  0.0
None
QB Interception Accuracy: 
correct answers:  0.285714285714
false answers:  0.714285714286
QB Passing Accuacy: 
correct answers:  0.642857142857
false answers:  0.357142857143
QB Touchdowns accuracy: 
correct answers:  0.857142857143
false answers:  0.142857142857
None
QB Interception Accuracy: 
correct answers:  0.4
false answers:  0.6
QB Passing Accuacy: 
correct answers:  0.933333333333
false answers:  0.0666666666667
QB Touchdowns accuracy: 
correct answers:  0.933333333333
false answers:  0.0666666666667
None


# RB Prediciton Accuracy

In [11]:

def RBTest(train, test):
    RBTrain = getRBStats(train)
    RBTrain
    RBProbability_Train = RBProbability(RBTrain)
#     print RBProbability_Train
    RBTest = getRBStats(test)
    RBProbability_Test = RBProbability(RBTest)
#     print RBProbability_Test

    l = []
    for i in range(len(RBProbability_Train.iloc[:,0])):
        for j in range(len(RBProbability_Test.iloc[:,0])):
            #age
                if(RBProbability_Train.iloc[i,0] == RBProbability_Test.iloc[j,0]):
                    #stat
                    if RBProbability_Train.iloc[i,1] == RBProbability_Test.iloc[j,1]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'RB Rushing Yards Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)

    l = []
    for i in range(len(RBProbability_Train.iloc[:,0])):
        for j in range(len(RBProbability_Test.iloc[:,0])):
            #age
                if(RBProbability_Train.iloc[i,0] == RBProbability_Test.iloc[j,0]):
                    #stat
                    if RBProbability_Train.iloc[i,2] == RBProbability_Test.iloc[j,2]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'RB Touchdown Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)

print RBTest(train17,test17)
print RBTest(train16,test16)
print RBTest(train15,test15)   

RB Rushing Yards Accuracy: 
correct answers:  0.916666666667
false answers:  0.0833333333333
RB Touchdown Accuracy: 
correct answers:  0.833333333333
false answers:  0.166666666667
None
RB Rushing Yards Accuracy: 
correct answers:  0.727272727273
false answers:  0.272727272727
RB Touchdown Accuracy: 
correct answers:  0.727272727273
false answers:  0.272727272727
None
RB Rushing Yards Accuracy: 
correct answers:  0.636363636364
false answers:  0.363636363636
RB Touchdown Accuracy: 
correct answers:  0.727272727273
false answers:  0.272727272727
None


# WR Predicaiton Accuracy

In [12]:
train = train17
test = test17
def WRTest(train, test):
    WRTrain = getWRStats(train)
    WRTrain
    WRProbability_Train = WRProbability(WRTrain)
#     print WRProbability_Train
    WRTest = getWRStats(test)
    WRProbability_Test = WRProbability(WRTest)
#     print WRProbability_Test

    # print WRProbability_Test.columns

    l = []
    for i in range(len(WRProbability_Train.iloc[:,0])):
        for j in range(len(WRProbability_Test.iloc[:,0])):
            #age
                if(WRProbability_Train.iloc[i,0] == WRProbability_Test.iloc[j,0]):
                    #stat
                    if WRProbability_Train.iloc[i,1] == WRProbability_Test.iloc[j,1]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'WR Recieivng Yards Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)

    l = []
    for i in range(len(WRProbability_Train.iloc[:,0])):
        for j in range(len(WRProbability_Test.iloc[:,0])):
            #age
                if(WRProbability_Train.iloc[i,0] == WRProbability_Test.iloc[j,0]):
                    #stat
                    if WRProbability_Train.iloc[i,2] == WRProbability_Test.iloc[j,2]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'WR Touchdown Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)
    
print WRTest(train17,test17)
print WRTest(train16,test16)
print WRTest(train15,test15)    

WR Recieivng Yards Accuracy: 
correct answers:  0.769230769231
false answers:  0.230769230769
WR Touchdown Accuracy: 
correct answers:  0.923076923077
false answers:  0.0769230769231
None
WR Recieivng Yards Accuracy: 
correct answers:  0.928571428571
false answers:  0.0714285714286
WR Touchdown Accuracy: 
correct answers:  0.857142857143
false answers:  0.142857142857
None
WR Recieivng Yards Accuracy: 
correct answers:  0.75
false answers:  0.25
WR Touchdown Accuracy: 
correct answers:  0.6875
false answers:  0.3125
None


# TE Predicaiton Accuracy

In [13]:
# train = createTable(players_df,train_df)
# test = createTable(players_df,test_df)
def TETest(train, test):
    TETrain = getTEStats(train)
    TETrain
    TEProbability_Train = TEProbability(TETrain)
#     print TEProbability_Train
    TETest = getTEStats(test)
    TEProbability_Test = TEProbability(TETest)
#     print TEProbability_Test

    # print TEProbability_Test.columns
    l = []
    for i in range(len(TEProbability_Train.iloc[:,0])):
        for j in range(len(TEProbability_Test.iloc[:,0])):
            #age
                if(TEProbability_Train.iloc[i,0] == TEProbability_Test.iloc[j,0]):
                    #stat
                    if TEProbability_Train.iloc[i,1] == TEProbability_Test.iloc[j,1]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'TE Recieving Yards Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)

    l = []
    for i in range(len(TEProbability_Train.iloc[:,0])):
        for j in range(len(TEProbability_Test.iloc[:,0])):
            #age
                if(TEProbability_Train.iloc[i,0] == TEProbability_Test.iloc[j,0]):
                    #stat
                    if TEProbability_Train.iloc[i,2] == TEProbability_Test.iloc[j,2]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'TE Touchdowns Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)
    
print TETest(train17,test17)
print TETest(train16,test16)
print TETest(train15,test15)    

TE Recieving Yards Accuracy: 
correct answers:  0.916666666667
false answers:  0.0833333333333
TE Touchdowns Accuracy: 
correct answers:  1.0
false answers:  0.0
None
TE Recieving Yards Accuracy: 
correct answers:  0.833333333333
false answers:  0.166666666667
TE Touchdowns Accuracy: 
correct answers:  1.0
false answers:  0.0
None
TE Recieving Yards Accuracy: 
correct answers:  0.666666666667
false answers:  0.333333333333
TE Touchdowns Accuracy: 
correct answers:  0.916666666667
false answers:  0.0833333333333
None


# K Predicaiton Accuracy

In [14]:
# train = createTable(players_df,train_df)
# test = createTable(players_df,test_df)

def KTest(train, test):
    KTrain = getKStats(train)
    KTrain
    KProbability_Train = KProbability(KTrain)
    KTest = getKStats(test)
    KProbability_Test = KProbability(KTest)

    l = []
    for i in range(len(KProbability_Train.iloc[:,0])):
        for j in range(len(KProbability_Test.iloc[:,0])):
            #age
                if(KProbability_Train.iloc[i,0] == KProbability_Test.iloc[j,0]):
                    #stat
                    if KProbability_Train.iloc[i,1] == KProbability_Test.iloc[j,1]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'Field Goals Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)

    l = []
    for i in range(len(KProbability_Train.iloc[:,0])):
        for j in range(len(KProbability_Test.iloc[:,0])):
            #age
                if(KProbability_Train.iloc[i,0] == KProbability_Test.iloc[j,0]):
                    #stat
                    if KProbability_Train.iloc[i,2] == KProbability_Test.iloc[j,2]:
                        l.append('y')
                    else:
                        l.append('f')

    T = float(l.count('y'))
    F = float(l.count('f') )
    print 'PAT Accuracy: '
    print 'correct answers: ', T / len(l)
    print 'false answers: ', F / len(l)
    
print KTest(train17,test17)
print KTest(train16,test16)
print KTest(train15,test15)    

Field Goals Accuracy: 
correct answers:  0.571428571429
false answers:  0.428571428571
PAT Accuracy: 
correct answers:  0.571428571429
false answers:  0.428571428571
None
Field Goals Accuracy: 
correct answers:  0.625
false answers:  0.375
PAT Accuracy: 
correct answers:  0.625
false answers:  0.375
None
Field Goals Accuracy: 
correct answers:  0.666666666667
false answers:  0.333333333333
PAT Accuracy: 
correct answers:  0.666666666667
false answers:  0.333333333333
None


In [43]:
def fantasyPoints(position, statRanges):
    passingyrds = 1.0/25.0
    passsingTD = 4.0
    interception = -2
    rushingyrds = 1.0/10
    rushingTD = 6
    recievingyrds = 1.0/10
    recievingTD = 6
    pat = 1
    fg = 3       
    
    if position == 'QB':
        INT = tuple([round((interception * i),2) for i in statRanges[0]])
        PS = tuple([round((passingyrds * i),2) for i in statRanges[1]])
        TD = tuple([round((passsingTD * i),2) for i in  statRanges[2]])
        
        row = {"Interceptions" : statRanges[0], "Passing Yards" : statRanges[1], "Passing Touchdowns" : statRanges[2], "Fantasy Points": tuple(map(lambda x, y,z: x + y + z, INT, PS, TD))}
        return row
    elif position == 'RB':
        return project
    elif (position == 'WR') | (position == 'TE'):
        return project
    else:
        return project 


# fantasyPoints('QB', [5, 5000, 25])

In [34]:
def getRange(x, cl):
    x= x[x > 0]
    a = float(x.min())         
    b = float((x.mean()  / 2))   
    c = float(x.mean() + b)
    d = float(x.max() )
    
    if cl == 1:
        return (a,b)
    elif cl == 2:
        return (b,c)
    else: 
        return (c,d)

In [1]:
#Main
#Add
add = "Tom Brady"
#Drop
Drop = "Drew Brees"

split = add.split(" ")
add_player = train[(train["first_name"] == split[0]) & (train["last_name"] == split[1])]
position = add_player['position'].iloc[0]
age = add_player['age'].max()
l = []

if position == 'QB':
    QB =getQBStats(train17)
    table = QBProbability(QB)
    if  age == table.iloc[18, 0]:
        l = (table.iloc[18,1:]).tolist()
        ranges = [getRange(pd.to_numeric(QB['passing_interceptions']),l[0]), getRange(pd.to_numeric(QB['passing_yards']),l[1]), getRange(pd.to_numeric(QB['passing_touchdowns']),l[2])]
print fantasyPoints(position,ranges)       


NameError: name 'train' is not defined

In [18]:
# # print pd.to_numeric(QB)

# print (QB['passing_yards'] > 0).min()

# getRange(pd.to_numeric(QB['passing_yards']),1)
# print pd.to_numeric(QB['passing_yards']).min()
# print pd.to_numeric(QB['passing_yards']).median() / 2
# print pd.to_numeric(QB['passing_yards']).median()
# print pd.to_numeric(QB['passing_yards']).median() + pd.to_numeric(QB['passing_yards']).median() / 2
# print pd.to_numeric(QB['passing_yards']).max()

# print 'mean'

# print pd.to_numeric(QB['passing_yards']).min()
# print pd.to_numeric(QB['passing_yards']).mean() / 2
# print pd.to_numeric(QB['passing_yards']).mean()
# print pd.to_numeric(QB['passing_yards']).mean() + pd.to_numeric(QB['passing_yards']).mean() / 2
# print pd.to_numeric(QB['passing_yards']).max()

True
-4
444.0
888.0
1332.0
6404
mean
-4
810.19297164
1620.38594328
2430.57891492
6404
